<!-- <div style='background-color: orange'> -->
<a id="TableOfContents"></a>
    <h1 style='text-align: center'>
        <b>
            Table of Contents:
        </b></h1>
<li><a href='#imports'>Imports</a></li>
<li><a href="#acquire">Acquire</a></li>
<li><a href='#prepare'>Prepare</a></li>
<li><a href="#wrangle">Wrangle</a></li>
<li><a href='#misc'>Miscellaneous</a></li>

<!-- <div style='background-color: orange'> -->
<a id="imports"></a>
    <h1 style='text-align: center'>
        <b>
            Imports
        </b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [2]:
# import data manipulation tools
import numpy as np
import pandas as pd
# import file access tools
import os
import requests
# import train_test_split function
from sklearn.model_selection import train_test_split
# import variable encoding function
from sklearn.preprocessing import LabelEncoder

# import env file containing username and password for database connection
# import env
import sample_env

<!-- <div style='background-color: orange'> -->
<a id="acquire"></a>
    <h1 style='text-align: center'>
        <b>
                Acquisition
        </b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

Acquire data via SQL

In [ ]:
def acquire_sql_data():
    '''
    this will read in the dataset from csv file in the local directory,
    if one does not exist, it will download the data via sql from the server assigned
    in the env file
    '''
    # set file name
    filename = 'sample_file.csv'
    # check if file exists in local directory
    if os.path.exists(filename):
        print('opening file from local directory')
        # read in the file to a dataframe
        df = pd.read_csv(filename, 
                         index_col=0, # set the index to the first column
                         
                         )
    # if the file doesn't exist
    else:
        # print an error
        print(f'file {filename} not found in local directory, downloading via SQL')
        # download dataset from codeup server
        # set database scheme we are looking at
        database = 'db_schema' 
        # create sql connectioni
        connection = env.get_db_url(database)
        # create sql query
        query = '''
select *
from schema
;
        '''
        # query server via sql
        df = pd.read_sql(query, connection)
        # cache the data to local csv file
        df.to_csv(filename)
    # return the dataframe
    return df

Acquire data from API

In [ ]:
def get_data(url, offset=1000, max_pages=200):
    '''
    this function will return a dataFrame with results from the passed api url
    and loop through the data until the last page of results, or after 200 pages
    '''
    print('downloading data via api')
    # create request.get for passed url
    page = requests.get(url)
    # create a dataframe withe the first page results
    df = pd.DataFrame(page.json())
    # set first offset amount
    x=offset
    #
    print
    # loop through all pages of results for passed url
    while page.json():
        # set the page url to the next page url
        next_url = url + f'&$offset={x}'
        page = requests.get(next_url)
        # add current page data to the dataframe
        df = pd.concat([df, pd.DataFrame(page.json())], axis=0)
        x += offset
        # display current number of rows collected
        print(f'{x}')
        # break the loop if max number of pages is reached
        if x >= offset * max_pages:
            break
    # display how many rows were downloaded
    print(f'{df.shape[0]} records were downloaded via api')
    # return the datframe
    return df

In [ ]:
def acquire_api_data():
    '''
    This function will read in datasets df1 and df2 data via csv,
    if the datasets don't exist in the local directory it will 
    it will then cache the data into csv files and return the dfs.
    '''
    # set the filenames we are looking for in the local directory
    df1_filename = 'sample_df1.csv'
    df2_filename = 'sample_df2.csv'
    # set the api paths
    token = 'api_token'
    df1_api = f'https://data.youWant.com/api/api1Path.json?$$app_token={token}'
    df2_api = f'https://data.youWant.com/api/api2Path.json?$$app_token={token}'
    
    # check if df1 files exist in the local directory
    if os.path.exists(df1_filename):
        # read in csv files for df1 dataset
        print('reading df1 dataset from local file')
        df1 = pd.read_csv(df1_filename, index_col=0)
    else:
        # if the df1 file does not exist locally, then download the data via api
        print('df1 data not found in local directory')
        df1 = get_data(df1_api)
        df1.to_csv(df1_filename, index=False)
        
    # check if df2 file exist in the local directory
    if os.path.exists(df2_filename):
        # read in csv files for df2 for second dataset
        print('reading df2 data from local file')
        df2 = pd.read_csv(df2_filename, index_col=0)
    else:
        # if the df2 file does not exist locally, then download the data via api
        print('df2 data not found in local directory')
        df2 = get_data(df2_api)
        df2.to_csv(df2_filename)
    
    # return the combined dataframe
    return df1, df2

<!-- <div style='background-color: orange'> -->
<a id="prepare"></a>
    <h1 style='text-align: center'>
        <b>
            Prepare
        </b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

Sample data preparation steps

In [ ]:
def prepare_data(df1, df2=None):
    '''
    this will prepare the dataframe(s) for use
    '''
    
    # merge dataframes into one
#     df = pd.merge(left=df1, right=df2, how='inner', 
#                        on='id', suffixes=('_df1','_df2'))
    
    # rename columns to lowercase, replace spaces and any . characters with _
    df.columns = [col.lower().\
                  replace(' ', '_').\
                  replace('.', '_') for col in df.columns]
    
    # change a datetime column to datetime type
#     df.datetime = pd.to_datetime(df.datetime)
    #set datetime as the index
#     df = df.set_index('datetime')

    # rename columns
#     df = df.rename(columns={'old_column_1':'new_column_1',
#                             'old_column_2':'new_column_2',
#                             'old_column_3':'new_column_3'})
    
    # create a new column with binned info
#     df['new_col'] = np.where(df.outcome_type == 'new_bin_value_1', 'old_value_1',
#                        np.where(df.outcome_type == 'new_bin_value_1', 'old_value_2',
#                        np.where(df.outcome_type == 'new_bin_value_2', 'old_value_3',
#                                )))

    # encode categorical variables
    
#     # convert first part of website path into column called lesson
#     # create an empty list
#     page_list=[]
#     # split the webpage path by '/'
#     pages = df.path.str.split('/')
#     # go through all of the pages
#     for i in range(len(pages)):
#         # make a list of the first part of the paths
#         page_list.append(pages[i][0])
#     # create a new column with the first part of the path
#     df['lesson'] = page_list

#     # return the prepared dataframe
    return df

Encode categorical variables

In [ ]:
def encode_cat_variables(df, encode_cols):
    # encode categorical columns so we can use them during explore and modeling
    # make a list of columns to encode
    encode_cols = ['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake',
               'breed', 'color', 'sex_upon_outcome', 'outcome_subtype']
    # create encoder object
    le = LabelEncoder()
    for col in encode_cols:
        le.fit(df[col])
        # create a new column with the encoded values
        df[f'{col}_encoded'] = le.transform(df[col])
    # we want to one-hot encode the outcome variable since that is our target
    df = pd.concat([df, pd.get_dummies(df.outcome)], axis=1)

<!-- <div style='background-color: orange'> -->
<a id="wrangle"></a>
    <h1 style='text-align: center'>
        <b>
            Wrangle
        </b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [ ]:
def wrangle_web_traffic():
    '''
    this will perform both acquire and preparation steps in one command
    '''
    # acquire and prepare data
    df = prepare_data(acquire_sql_data())
    # split data into train, validate and test groups
    train, validate, test = split_data(df)
    # return the original and split dfs
    return df, train, validate, test

<!-- <div style='background-color: orange'> -->
<a id="misc"></a>
    <h1 style='text-align: center'>
        <b>
            Miscellaneous
        </b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>